# Transformation of the "Rapporti di lavoro attivati" and "Rapporti di lavoro prorogati" datasets.
#### The input and output datasets can be found in the `DatasetsLocation.md` file in the `Datasets` folder.

If you want to actually execute the instructions, it is necessary to insert the path of the input and output dataset. Then, the notebook can be run altogether.

In [ ]:
activatedContractsPath = "<input_path>\\Rapporti_di_lavoro_attivati.csv"
activatedTranformedPath = "<output_path>\\Transformed_lavori_attivati.csv"
extendedContractsPath = "<input_path>\\Rapporti_di_lavoro_prorogati.csv"
extendededTranformedPath = "<output_path>\\Transformed_lavori_prorogati.csv"

All the necessary libraries are imported:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

Create the data frames from the csv files

In [ ]:
activatedContracts = pd.read_csv(activatedContractsPath)
'''create a backup of the original for confrontations later on'''
originalActivatedContracts = activatedContracts.copy(deep=True)
activatedContracts.head()

In [ ]:
extendedContracts = pd.read_csv(extendedContractsPath)
'''create a backup of the original for confrontations later on'''
originalExtendedContracts = extendedContracts.copy(deep=True)
extendedContracts.head()

In [ ]:
print("activatedContracts shape: ",activatedContracts.shape)
print("extendedContracts shape: ",extendedContracts.shape)

In [ ]:
print("activatedContracts dtypes:\n",activatedContracts.dtypes, sep="")
print("")
print("extendedContracts dtypes:\n",extendedContracts.dtypes, sep="")

The type of the DATA column is changed in a Date format.

In [ ]:
activatedContracts.DATA = activatedContracts.DATA.astype(np.datetime64)
activatedContracts.DATA 

In [ ]:
extendedContracts.DATA = extendedContracts.DATA.astype(np.datetime64)
extendedContracts.DATA

In activatedContracts, there is one tuple that is set in the future (and in general after 2021). Considering the year, 
this is probably just a typo so we can correct it. 

In [ ]:
futureDate = activatedContracts[activatedContracts.DATA > "2021/12/31"]
futureDate

In [ ]:
activatedContracts.loc[activatedContracts.DATA > "2021-12-31", ["ETA"]] = 221-(2201-2021)
activatedContracts.loc[activatedContracts.DATA > "2021-12-31", ["DATA"]] = np.datetime64("2021-06-09")
len(activatedContracts[activatedContracts.DATA > "2021-12-31"])

In [ ]:
'''I know that the index is 5840160 and will call it directly to check if the transformation is correct'''
activatedContracts.iloc[5840160]

In extendedContracts, there is no tuples after the 31st of December 2021.

In [ ]:
len(extendedContracts[extendedContracts.DATA > "2021-12-31"])

By reading the data, and as shown in the following histograms, the data seems to be incomplete for the period before 2009.

The extended Contracts dataset only has very few tuples for the year 2008, while the activated contracts dataset has a small number
of tuples between 1973 and 2008.

In [ ]:
totalDates_activated = activatedContracts.DATA.astype('datetime64[Y]')
totalDates_activated = totalDates_activated.groupby(totalDates_activated).count()
totalDates_activated.index = totalDates_activated.index.year

totalDates_extended = extendedContracts.DATA.astype('datetime64[Y]')
totalDates_extended = totalDates_extended.groupby(totalDates_extended).count()
totalDates_extended.index = totalDates_extended.index.year

fig = plt.figure(figsize=(20, 7))

act = fig.add_subplot(121)
act.title.set_text('Activated contracts per year')
totalDates_activated.plot(kind="bar", xlabel='')

act = plt.subplot(122)
act.title.set_text('Extended contracts per year')
totalDates_extended.plot(kind="bar", xlabel='')

plt.show()

In [ ]:
print("Number of activated contracts registered in 2008: ",
      len(activatedContracts[activatedContracts.DATA.between('2008-01-01','2008-12-31')]))

print("Number of activated contracts registered in 2007: ",
      len(activatedContracts[activatedContracts.DATA.between('2007-01-01','2007-12-31')]))

print("Number of extended contracts registered in 2008: ",
      len(extendedContracts[extendedContracts.DATA.between('2008-01-01','2008-12-31')]))

Because of these results, I would exclude all contracts that have been activated before the year 2008. 
I will keep this last year because some of the contracts that have been extended in 2009 were supposedly activated in 2008.

In [ ]:
contractsBefore2008 = len(activatedContracts[activatedContracts.DATA.between('1960-01-01','2007-12-31')])
print("Number of activated contracts before 2008: ",contractsBefore2008)

In [ ]:
originalLength = len(activatedContracts)
activatedContracts = activatedContracts[activatedContracts.DATA.between('2008-01-01','2021-12-31')]
print("Number of dropped rows: ",originalLength - len(activatedContracts))

Since the two datasets come from the same source, they consider the same types of contracts. In our test case, we only need
to know which type of contracts are extendable and which are not. We can consider all the contract types in the extendedContracts
dataframe to be "extendable", making the rest "not extendable". 

The values for the column "CONTRATTO" in activatedContracts will
be changed into "PROROGABILE" and "NON PROROGABILE".

This is not be done for extendedContracts because all values would be "PROROGABILE" and in general could be useful in the future.

In [ ]:
extendableContractsList = extendedContracts.CONTRATTO.unique()
condition = activatedContracts.CONTRATTO.isin(extendableContractsList)
activatedContracts.loc[-condition,'CONTRATTO'] = "NON PROROGABILE"
activatedContracts.loc[condition,'CONTRATTO'] = "PROROGABILE"

activatedContracts[activatedContracts.CONTRATTO != "PROROGABILE"]

When talking about italian labour market, we are usually interested only in dividing italians from foreigners. For this reason,
the "ITALIANO" column is substituted by a "CITTADINANZA" column containing the values "ITALIANO" and "STRANIERO"

In [ ]:
activatedContracts.rename(columns={"ITALIANO":"CITTADINANZA"}, inplace=True)
condition = activatedContracts.CITTADINANZA == "ITALIA"
activatedContracts.loc[-condition, 'CITTADINANZA'] = "STRANIERO"
activatedContracts.loc[condition, 'CITTADINANZA'] = "ITALIANO"
activatedContracts

Now the same thing for extendedContracts

In [ ]:
extendedContracts.rename(columns={"ITALIANO":"CITTADINANZA"},inplace=True)
condition = extendedContracts.CITTADINANZA == "ITALIA"
extendedContracts.loc[-condition, 'CITTADINANZA'] = "STRANIERO"
extendedContracts.loc[condition, 'CITTADINANZA'] = "ITALIANO"
extendedContracts

Since we want to consider the education degree of a person, we check whether the values are the same in the two datasets:

In [ ]:
activatedContracts.TITOLOSTUDIO.unique().tolist()

In [ ]:
extendedContracts.TITOLOSTUDIO.unique().tolist()

The values are the same, but we can see that some tuples have no value at all. It may be possible to predict the degree by
analyzing the values of the other attributes (via means and modes), but I think it is better to just eliminate them.

In [ ]:
conditionA = activatedContracts.TITOLOSTUDIO.isna()
conditionE = extendedContracts.TITOLOSTUDIO.isna()
print("Number of tuples without a 'TITOLOSTUDIO' value")
print("For activatedContracts: ", len(activatedContracts[conditionA]))
print("For extendedContracts: ", len(extendedContracts[conditionE]))

In [ ]:
originalLengthA = len(activatedContracts)
originalLengthE = len(extendedContracts)
activatedContracts = activatedContracts[-conditionA]
extendedContracts = extendedContracts[-conditionE]
print("Number of dropped rows in activatedContrats: ", originalLengthA-len(activatedContracts))
print("Number of dropped rows in extendedContracts: ", originalLengthE-len(extendedContracts))

Since I also want to classify people by gender and age, I check for invalid values for GENERE and ETA

In [ ]:
activatedContracts.GENERE.unique().tolist()

In [ ]:
extendedContracts.GENERE.unique().tolist()

In [ ]:
activatedContracts.ETA.describe()

In [ ]:
extendedContracts.ETA.describe()

For the ETA attribute, the values range from 0 to 100(or 101 depending on the dataset).

In [ ]:
activatedContracts[activatedContracts.ETA < 15]['ETA'].unique().tolist()

In [ ]:
extendedContracts[extendedContracts.ETA < 15]['ETA'].unique().tolist()

For the same reasons explained for the education degree attribute, I would drop all tuples with an age lower than 15. 

In [ ]:
conditionA = activatedContracts.ETA < 15
conditionE = extendedContracts.ETA < 15
print("Number of tuples without a 'ETA' value lower than 15")
print("For activatedContracts: ", len(activatedContracts[conditionA]))
print("For extendedContracts: ", len(extendedContracts[conditionE]))

In [ ]:
originalLengthA = len(activatedContracts)
originalLengthE = len(extendedContracts)
activatedContracts = activatedContracts[-conditionA]
extendedContracts = extendedContracts[-conditionE]
print("Number of dropped rows in activatedContrats: ", originalLengthA-len(activatedContracts))
print("Number of dropped rows in extendedContracts: ", originalLengthE-len(extendedContracts))

In total, we removed `66 + 580 + 12162 = 12808` rows  from activatedContracts, which equals to the 0.13% of the total 
rows of the original dataset, and `160 + 1284 = 1444` rows from extendedContracts, which is the 0.04% of the original datset.

Here below we have the percentage of removed row per education degree (the first list refers to activatedContracts and the second to extendedContracts), which is lower than 1% for all cases in both datasets

In [ ]:
100 - ((activatedContracts.TITOLOSTUDIO.groupby(activatedContracts.TITOLOSTUDIO).count() / originalActivatedContracts.TITOLOSTUDIO.groupby(originalActivatedContracts.TITOLOSTUDIO).count())*100)

In [ ]:
100 - ((extendedContracts.TITOLOSTUDIO.groupby(extendedContracts.TITOLOSTUDIO).count() / originalExtendedContracts.TITOLOSTUDIO.groupby(originalExtendedContracts.TITOLOSTUDIO).count())*100)

In [ ]:
print("Final lenght of activatedContracts: ", len(activatedContracts))

print("Final lenght of extendedContracts: ", len(extendedContracts))

One last thing that could be done is remove the PROVINCIAIMPRESA and MODALITALAVORO since i do not intend to use them, but i will 
not do it because it may be of some use in the future.

The next istructions will export the new datasets as csv to the previously indicated paths. 

In [ ]:
activatedContracts.to_csv(activatedTranformedPath, index=False)
extendedContracts.to_csv(extendededTranformedPath, index=False)
print("done")